In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
SRC = '/Users/fathe/OneDrive/Documents/UK/MFF/Thesis/input'
videos = [file for file in os.listdir(SRC) if file.endswith("MP4") and file.startswith("GX")]
axis_horizontal = "y_shift"
axis_horizontal_diff = "y_shift_diff"
axis_vertical = "x_shift"
axis_vertical_diff = "x_shift_diff"

In [3]:
video = videos[0]

In [4]:
csvs = [file for file in os.listdir(SRC) if file.endswith("csv") and file.startswith(video.removesuffix('.MP4'))]

In [5]:
csvs

In [6]:
dfs = []
for csv in csvs:
    df = pd.read_csv(os.path.join(SRC, csv))
    dfs.append(df)

In [8]:
intervals_horizontal = np.load(os.path.join(SRC, video).replace(".MP4", "-intervals.npy"))
intervals_vertical = []
mask_horizontal = pd.Series(False, index=dfs[0].index)
mask_vertical = pd.Series(False, index=dfs[0].index)
end_last = None
for start, end in intervals_horizontal:
    mask_horizontal |= (dfs[0]['frame_ID'] >= start) & (dfs[0]['frame_ID'] <= end)
    
    if end_last is not None:
        mask_vertical |= (dfs[0]['frame_ID'] >= end_last) & (dfs[0]['frame_ID'] <= start)
        intervals_vertical.append((end_last, start))
    end_last = end

In [9]:
for df, csv in zip(dfs, csvs):
    df.loc[mask_horizontal, axis_vertical_diff] = 0
    df.loc[mask_vertical, axis_horizontal_diff] = 0
    df.to_csv(os.path.join(SRC, csv).replace('.csv', '-preprocessed.csv'), index=False)